#1 Install dependencies and import libraries

In [1]:
!pip install basicsr -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.0/303.0 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.2/256.2 kB 16.1 MB/s eta 0:00:00


In [2]:
#basicssr contains a file degradation.py where the import library is obsolete
#and has to be replaced
#https://github.com/xinntao/Real-ESRGAN/issues/801

file_path = "/usr/local/lib/python3.12/dist-packages/basicsr/data/degradations.py"
line_to_replace = "from torchvision.transforms.functional_tensor import rgb_to_grayscale"
new_line = "from torchvision.transforms.functional import rgb_to_grayscale"

with open(file_path, 'r') as file:
    lines = file.readlines()

lines = [new_line if line.strip() == line_to_replace else line for line in lines]

with open(file_path, 'w') as file:
    file.writelines(lines)

print(f"Replacing line '{line_to_replace}' with '{new_line}' in file {file_path}")

Replacing line 'from torchvision.transforms.functional_tensor import rgb_to_grayscale' with 'from torchvision.transforms.functional import rgb_to_grayscale' in file /usr/local/lib/python3.12/dist-packages/basicsr/data/degradations.py


In [30]:
import os
import cv2
import matplotlib.pyplot as plt
import ipywidgets as widgets
from skimage import io, metrics, transform, color
import basicsr
from basicsr.metrics.niqe import calculate_niqe as bs_calculate_niqe
from PIL import Image
import numpy as np
import zipfile
import pandas as pd

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 2 Calculate PSNR

In [28]:
def calculate_psnr(gt_path, test_path):
    """
    Calculate PSNR between two images. Assumes both images have same dimensions.
    """
    # Read images
    gt_image = io.imread(gt_path)
    test_image = io.imread(test_path)

    # Remove alpha channel if present
    if gt_image.shape[2] == 4:
        gt_image = gt_image[:, :, :3]
    if test_image.shape[2] == 4:
        test_image = test_image[:, :, :3]

    # Calculate PSNR
    psnr_value = metrics.peak_signal_noise_ratio(gt_image, test_image)
    return psnr_value

# 3 Calculate SSIM

In [16]:
def calculate_ssim(gt_path, test_path):
    """
    Calculate SSIM between two images. Assumes both images have same dimensions.
    """
    # Read images
    gt_image = io.imread(gt_path)
    test_image = io.imread(test_path)

    # Remove alpha channel if present
    if gt_image.shape[2] == 4:
        gt_image = gt_image[:, :, :3]
    if test_image.shape[2] == 4:
        test_image = test_image[:, :, :3]

    # Calculate SSIM (1 = perfect match)
    ssim_value = metrics.structural_similarity(gt_image, test_image,
                                               channel_axis=2,
                                               data_range=255)
    return ssim_value

# 4 Calculate NIQE

In [17]:
def niqe_from_path(image_path, crop_border=0):
    """
    Calculate NIQE for a single image using BasicSR in grayscale
    with consistent scaling for better alignment with perception.
    """
    # Open and convert to grayscale
    img = Image.open(image_path).convert('L')  # L = grayscale
    img_np = np.array(img).astype(np.float32)

    # Scale image to 0-255 if not already
    if img_np.max() <= 1.0:
        img_np *= 255.0

    # Add channel dimension: HWC
    img_np = img_np[:, :, np.newaxis]

    # Calculate NIQE
    return bs_calculate_niqe(img_np, crop_border=crop_border, input_order='HWC')


#5 get metrics for images inferred from fine-tuned and non-fine tuned model

In [12]:
# Paths for ground truth and inferred images
gt_zip = "/content/drive/MyDrive/Final_project/gt_test_images.zip"
models_zip = "/content/drive/MyDrive/Final_project/models_test_results.zip"

gt_dir = "/content/models_test_results/"
models_dir = "/content/models_test_results/"

# Unzip ground truth
with zipfile.ZipFile(gt_zip, 'r') as zip_ref:
    zip_ref.extractall(gt_dir)

# unzip model results
with zipfile.ZipFile(models_zip, 'r') as zip_ref:
    zip_ref.extractall(models_dir)

In [54]:
# Choose model and metric
option = input("Enter 1 for Real-ESRGAN, 2 for HAT, 3 for SwinIR and 4 for BSRGAN: ")

if option == "1":
    model_name = "realesrgan"
elif option == "2":
    model_name = "hat"
elif option == "3":
    model_name = "swinir"
elif option == "4":
    model_name = "bsrgan"
else:
    raise ValueError("Invalid option. Please choose 1, 2, 3, or 4.")

metric_option = input("Enter 1 for PSNR, 2 for SSIM, 3 for NIQE: ")
metric_name = "PSNR" if metric_option == "1" else ("SSIM" if metric_option == "2" else "NIQE")

# Define folders and versions. versions will later correspond to csv header
categories = ["buildings", "desert", "snowregion", "vegetation", "water"]
versions = [
    ("FineTuned", f"finetuned_{metric_name.lower()}"),
    ("NotFineTuned", f"non_finetuned_{metric_name.lower()}")
]

# Add suffix condition
if model_name == "realesrgan":
    suffix = "_out.png"
elif model_name == "hat":
    suffix = "_HAT-L_SRx4_ImageNet-pretrain.png"
elif model_name == "swinir":
    suffix = "_SwinIR.png"
elif model_name == "bsrgan":
    suffix = "_BSRGAN.png"
else:
    suffix = ".png"  # fallback


Enter 1 for Real-ESRGAN, 2 for HAT, 3 for SwinIR and 4 for BSRGAN: 2
Enter 1 for PSNR, 2 for SSIM, 3 for NIQE: 3


In [55]:
results = []

# Loop through categories and versions-
for folder in categories:
    for version_folder, metric_col in versions:
        zip_path = os.path.join(models_dir, model_name, version_folder, f"results_{folder}.zip")
        extract_path = os.path.join("/tmp", f"{model_name}_{version_folder}_{folder}")
        os.makedirs(extract_path, exist_ok=True)

        if not os.path.exists(zip_path):
            print(f"Zip not found: {zip_path}")
            continue

        #unzip categories folder from models_results
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)

        for fname in os.listdir(extract_path):
            if not fname.endswith(suffix):
                continue

            inferred_image_path = os.path.join(extract_path, fname)
            gt_image_name = fname.replace(suffix, ".png")
            gt_image_path = os.path.join(gt_dir, folder, gt_image_name)

            if not os.path.exists(gt_image_path):
                print(f"GT missing for {fname}")
                continue

            if metric_option == "1":
                metric_value = calculate_psnr(gt_image_path, inferred_image_path)
            elif metric_option == "2":
                metric_value = calculate_ssim(gt_image_path, inferred_image_path)
            elif metric_option == "3":
                metric_value = niqe_from_path(inferred_image_path)

            entry = next((r for r in results if r["image_name"] == fname), None)
            if entry:
                entry[metric_col] = metric_value
            else:
                results.append({"image_name": fname, "category": folder, metric_col: metric_value})

# Save results to CSV
df_results = pd.DataFrame(results).fillna("")
csv_path = f"/content/{metric_name.lower()}_{model_name}_results.csv"
df_results.to_csv(csv_path, index=False)
print(f"Saved results to {csv_path}")

Saved results to /content/niqe_hat_results.csv


# 6 Get average of metrics

In [56]:
#!!Important: modify the model_name correct one
eval_metrics = ["psnr", "ssim", "niqe"]
model_name = "realesrgan"

for metric in eval_metrics:
    csv_path = f"/content/{metric}_{model_name}_results.csv"
    df = pd.read_csv(csv_path)

    finetuned_col = f"finetuned_{metric}"
    non_finetuned_col = f"non_finetuned_{metric}"

    if finetuned_col in df.columns and non_finetuned_col in df.columns:
        avg_df = df.groupby("category")[[finetuned_col, non_finetuned_col]].mean()
        print(f"\nAverage {metric.upper()} per category for {model_name}:")
        print(avg_df)
    else:
        print(f"\nColumns for metric '{metric}' not found in {csv_path}")



Average PSNR per category for realesrgan:
            finetuned_psnr  non_finetuned_psnr
category                                      
buildings        14.487555           14.869702
desert           17.478855           18.076298
snowregion       17.828872           18.370952
vegetation       19.119449           18.934369
water            20.778578           20.914341

Average SSIM per category for realesrgan:
            finetuned_ssim  non_finetuned_ssim
category                                      
buildings         0.320293            0.289811
desert            0.355033            0.345877
snowregion        0.413375            0.380883
vegetation        0.398424            0.330023
water             0.674964            0.655344

Average NIQE per category for realesrgan:
            finetuned_niqe  non_finetuned_niqe
category                                      
buildings         5.925376            5.090339
desert            5.569431            5.977473
snowregion        5.16053

# 7 Difference between metrics

In [57]:
#!!Important: modify the model_name correct one
models = ["realesrgan"]
eval_metrics = ["psnr", "ssim", "niqe"]

# Loop over models and eval_metrics
for model_name in models:
    for metric in eval_metrics:
        csv_path = f"/content/{metric}_{model_name}_results.csv"
        df = pd.read_csv(csv_path)

        # Column names
        finetuned_col = f"finetuned_{metric}"
        non_finetuned_col = f"non_finetuned_{metric}"
        diff_col = f"diff_{metric}"

        # Compute difference: positive = fine-tuned better, negative = non-fine-tuned better
        if metric == "niqe":
            df[diff_col] = df[non_finetuned_col] - df[finetuned_col]
        else:
            df[diff_col] = df[finetuned_col] - df[non_finetuned_col]

        # Save updated CSV
        df.to_csv(csv_path, index=False)
        print(f"\nUpdated CSV saved with {diff_col} for {model_name}")

        # Compute 2nd percentile threshold for "non-finetuned better" cases
        threshold = df[diff_col].quantile(0.02)

        # Output worst cases (non-finetuned better)
        worst_cases = df[df[diff_col] <= threshold]
        print(f"\nTop non-finetuned better images for {metric.upper()} - {model_name}:")
        print(worst_cases)



Updated CSV saved with diff_psnr for realesrgan

Top non-finetuned better images for PSNR - realesrgan:
                   image_name category  finetuned_psnr  non_finetuned_psnr  \
62    desert2_patch_6_out.png   desert       13.598085           15.047684   
63   desert2_patch_23_out.png   desert       15.215955           16.674884   
74   desert2_patch_26_out.png   desert       15.733445           17.372210   
91   desert2_patch_22_out.png   desert       16.490226           18.084324   
99   desert2_patch_12_out.png   desert       13.795557           15.659758   
102   desert2_patch_5_out.png   desert       12.647449           14.579856   

     diff_psnr  
62   -1.449600  
63   -1.458929  
74   -1.638765  
91   -1.594098  
99   -1.864202  
102  -1.932407  

Updated CSV saved with diff_ssim for realesrgan

Top non-finetuned better images for SSIM - realesrgan:
                   image_name category  finetuned_ssim  non_finetuned_ssim  \
61   desert2_patch_19_out.png   desert        